In [6]:
import numpy as np
import os
from cloudvolume import CloudVolume, Skeleton
import navis
import pandas as pd
import logging
from google.cloud import storage

In [7]:
def generate_ngl_segmentation_empty(out_path):
    """Create an empty neuroglancer precomputed segmentation volume.
       out_path: directory where new the new cloud volume segmentation layer should be generated
    """
    
    info = CloudVolume.create_new_info(
        num_channels    = 1,
        layer_type      = 'segmentation',
        data_type       = 'uint64', # Channel images might be 'uint8'
        # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, compresso
        encoding        = 'compressed_segmentation', 
        resolution      = [0,0,0], # Voxel scaling, units are in nanometers
        voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
        chunk_size      = [0,0,0], # units are voxels
        volume_size     = [0,0,0], # e.g. a cubic millimeter dataset
        skeletons       = 'skeletons'
        )

    vol = CloudVolume(out_path, info=info, compress='', cache=False)
    logging.info(f"Creating cloud volume: {vol.info}")
    vol.commit_info()
    vol.commit_provenance()

In [8]:
inputdir = 'gs://allen-minnie-phase3/wqy_myelin/minnie/output_one/'
outdir = 'gs://allen-minnie-phase3/wqy_myelin/minnie/precomputed/'

In [10]:
# load csv file
rid = 864691135572530981
file_name = 'gs://allen-minnie-phase3/wqy_myelin/minnie/output_one/864691135572530981'

# Read the SWC files from Google Cloud Storage
swc_df = pd.read_csv(file_name)
swc_df

,node_id,parent_id,x,y,z,radius,type,normal_x,normal_y,normal_z,...,pixels_myelin_xy2d,pixels_outline_xy2d,myelin_metric_edt_xy2d,dis2myelin_xy2d,radius_xs2d,myelin_metric_xs2d,pixels_myelin_xs2d,pixels_outline_xs2d,myelin_metric_edt_xs2d,dis2myelin_xs2d
0,1038.0,7175.0,7.646560e+05,889632.000000,957920.000000,138.000000,end,-0.999739,0.020564,-0.009926,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[103, 108, 114, 120, 126, 132, 138, 144, 150, ...",0.000000,[21.095022 22.090721 23.086792 24.083189 20.22...,372.786493,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[78, 80, 83, 87, 91, 95, 99, 103, 107, 111]",0.000000,[59.20304 60.166435 61.13101 62.0967 63.06...
1,7175.0,7178.0,7.651731e+05,889698.698650,958147.016316,138.000000,slab,-0.999739,0.020564,-0.009926,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[108, 116, 123, 130, 138, 146, 153, 161, 169, ...",0.000000,[47.539455 47.92703 48.332184 48.754486 4...,127.677044,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[74, 73, 71, 72, 75, 76, 79, 83, 87, 91]",0.000000,[41.14608 42.047592 40.706264 43.382023 41.18...
2,7178.0,7181.0,7.662203e+05,889779.222747,958141.742614,138.000000,slab,-0.999739,0.020564,-0.009926,...,"[0, 0, 0, 0, 0, 0, 0, 3, 5, 7]","[165, 172, 181, 191, 201, 210, 220, 230, 233, ...",0.000000,[48.8467 49.24429 49.658836 50.08992 5...,146.967317,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.6666666666666667, ...","[0, 0, 0, 0, 0, 1, 2, 4, 2, 4]","[46, 45, 48, 52, 56, 60, 64, 68, 72, 76]",2.439024,[ 4.2426405 5. 5.8309517 6.708204 ...
3,7181.0,7184.0,7.676390e+05,889676.504030,958043.518178,138.000000,slab,-0.998234,0.055244,0.021859,...,"[0, 0, 2, 11, 13, 10, 11, 8, 7, 8]","[229, 229, 231, 233, 234, 234, 236, 238, 240, ...",6.410256,[26.683329 25.495098 25.96151 26.24881 2...,85.710004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.773...","[0, 0, 0, 0, 0, 0, 0, 0, 2, 3]","[22, 25, 29, 33, 37, 41, 45, 49, 53, 57]",0.000000,[14.21267 13.601471 13.038404 13.453624 1...
4,7184.0,7187.0,7.691238e+05,889561.614242,958072.804625,138.000000,slab,-0.998193,0.059637,0.007364,...,"[5, 7, 8, 17, 18, 16, 18, 14, 13, 15]","[245, 248, 252, 256, 259, 262, 263, 260, 260, ...",12.653061,[ 4.472136 5.3851647 6.3245554 7.28011 ...,101.156043,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[25, 28, 32, 36, 40, 44, 48, 52, 56, 60]",0.000000,[12.806249 13.453624 11.401754 12.727922 1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5061,22355.0,22358.0,1.355286e+06,861683.326477,837707.773646,366.000000,slab,-0.470755,0.871186,0.139373,...,"[253, 263, 263, 260, 222, 148, 117, 104, 88, 64]","[270, 270, 270, 272, 272, 273, 275, 276, 274, ...",97.163121,[1. 1. 1. 1. 1. ...,334.836551,"[100.0, 100.0, 98.88888888888889, 91.489361702...","[91, 88, 89, 86, 64, 42, 27, 24, 18, 10]","[91, 88, 90, 94, 98, 102, 106, 110, 114, 116]",100.000000,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...
5062,22358.0,22361.0,1.356114e+06,860513.939345,837470.188437,366.000000,slab,-0.615632,0.786047,0.055923,...,"[210, 212, 213, 214, 199, 137, 93, 70, 65, 60]","[238, 243, 247, 252, 255, 256, 260, 265, 268, ...",84.313725,[ 1. 1. 1. 1. ...,336.659644,"[83.14606741573034, 79.06976744186046, 82.9545...","[74, 68, 73, 65, 49, 34, 21, 17, 12, 8]","[89, 86, 88, 92, 96, 100, 104, 108, 112, 116]",96.428571,[1. 1. 1. 1. 1. ...
5063,22361.0,22364.0,1.356258e+06,859290.570884,836826.448753,366.000000,slab,-0.118341,0.914909,0.385924,...,"[101, 108, 104, 91, 65, 49, 40, 39, 28, 26]","[125, 133, 137, 133, 130, 131, 135, 137, 140, ...",94.017094,[1. 1. 1. 1. 1. ...,329.763608,"[71.56862745098039, 71.15384615384616, 65.1785...","[73, 74, 73, 60, 52, 36, 22, 8, 5, 4]","[102, 104, 112, 119, 127, 135, 143, 151, 159, ...",75.531915,[ 1. 1. 1. 1. 1. ...
5064,22364.0,1495.0,1.356071e+06,857997.100269,836142.983858,312.946364,slab,-0.168200,-0.865828,-0.471221,...,"[107, 114, 123, 118, 66, 43, 26, 19, 18, 1

In [ ]:
# summary of myelin metric

swc_df["max_myelin_metric_xy2d"] = swc_df["myelin_metric_xy2d"].apply(lambda x: max(map(float, x[1:-1].split(",")[:10])))
swc_df["max_myelin_metric_xy2d"]

0        0.000000
1        0.000000
2        2.953586
3        5.555556
4        6.949807
          ...    
5061    97.407407
5062    88.235294
5063    81.203008
5064    66.486486
5065     7.003891
Name: max_myelin_metric_xy2d, Length: 5066, dtype: float64

In [11]:
generate_ngl_segmentation_empty(outdir)

In [12]:
vol = CloudVolume(outdir, compress='')
# vol.skeleton.meta.info.pop("vertex_attributes", None)
sk_info = vol.skeleton.meta.default_info()
# Declare "orientation" as a 3-component vertex attribute
sk_info["vertex_attributes"] = [
    { 'id': 'radius',
        'data_type': 'float32',
        'num_components': 1
    }, 
    {
        "id": "myelin",
        "data_type": "float32",
        "num_components": 1,
        "description": "float number 0 or 1"
    }
]
vol.skeleton.meta.info = sk_info
vol.skeleton.meta.commit_info()
    
skel_dir = os.path.join(outdir, "skeletons")
if not os.path.exists(skel_dir):
    os.makedirs(skel_dir)



In [ ]:
if 'label' not in swc_df:
    swc_df.insert(1, 'label', list(np.zeros(len(swc_df.node_id))))
#reformat skeleton node table to swc
if isinstance(swc_df.label[0], str):
    swc_df.label=0
sk_nodes = swc_df[['node_id', 'label','x','y','z','radius','parent_id']]
sk_nodes[['node_id', 'label','parent_id']] = sk_nodes[['node_id', 'label','parent_id']].astype(int)
sk_nodes = list(list(x) for x in zip(*(sk_nodes[x].values.tolist() for x in sk_nodes.columns)))
sk_nodes = '\n'.join(str(x)[1:-1] for x in sk_nodes).replace(",", "")
skel = Skeleton.from_swc(sk_nodes)

# Add myelin component to vertex attributes
skel.extra_attributes.append({
    "id": "myelin",
    "data_type": "float32",
    "num_components": 1,
})

skel.myelin = swc_df['max_myelin_metric_xy2d'].values.astype('float32')

skel.id = str(rid)

# remove vertex_type from Skeleton
skel.extra_attributes = [ 
    attr for attr in skel.extra_attributes
    if attr["data_type"] == "float32"
  ]

# remove vertex_type from meta info
vol.skeleton.meta.info['vertex_attributes'] = [ 
    attr for attr in vol.skeleton.meta.info['vertex_attributes']
    if attr["data_type"] == "float32"
  ]

vol.skeleton.upload(skel)

/tmp/ipykernel_4119004/1911032666.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sk_nodes[['node_id', 'label','parent_id']] = sk_nodes[['node_id', 'label','parent_id']].astype(int)


In [15]:
skel

Skeleton(segid=864691135572530981, vertices=(shape=5066, float32), edges=(shape=5065, uint32), radius=(5066, float32), myelin=(5066, float32), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

##### https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/4765718475177984